In [1]:
import s3fs
import sys
sys.path.append('edit-project/util')
import util_data_access as da
from tqdm.auto import tqdm

# Download Files

In [2]:
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'http://s3.dev.obdc.bcs.bloomberg.com'})

In [3]:
for f in tqdm(fs.ls('aspangher/edit-pathways/spark_processing_scripts-output/nyt/')):
    f = '/'.join(f.split('/')[1:])
    fname = f.split('/')[-1]
    
    da.download_file('nyt_output/' + fname, f)

  0%|          | 0/784 [00:00<?, ?it/s]

In [ ]:
da.download_file('nyt.db', 'edit-pathways/dbs/newssniffer-nytimes.db')

In [4]:
fs.ls('aspangher/edit-pathways/spark_processing_scripts-output_sentences/nyt')

['aspangher/edit-pathways/spark_processing_scripts-output_sentences/nyt/df_nyt__start_0__end_20000__num_1.pkl']

# Read Files

In [5]:
import os, glob
import pandas as pd 
dfs = []
for f in glob.glob('nyt_output/*'):
    df = pd.read_csv(f, compression=None, index_col=0)
    dfs.append(df)

In [6]:
full_diffs_df = pd.concat(dfs)

In [8]:
full_diffs_df[['entry_id', 'version_x', 'version_y']].drop_duplicates().shape

(49904, 3)

In [10]:
full_diffs_df.head()

,entry_id,version_x,version_y,sent_idx_x,sent_idx_y,avg_sentence_distance_x,avg_sentence_distance_y
0,566535,2,3,NaN,50.0,NaN,NaN
1,565936,1,2,27.0,27.0,0.0,0.0
2,566126,0,1,210.0,210.0,0.0,0.0
3,566639,0,1,50.0,50.0,0.0,0.0
4,566157,3,4,NaN,34.0,NaN,NaN


In [28]:
full_diffs_df.assign(c=1).groupby(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])['c'].sum().value_counts()

1    1822246
2      81357
Name: c, dtype: int64

In [13]:
(full_diffs_df
 .fillna('nan')
 .drop_duplicates(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])
 .assign(c=1)
 .groupby(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])['c']
 .sum()
 .loc[lambda s: s>1]
#  .head()
)

Series([], Name: c, dtype: int64)

In [ ]:
import numpy as np

In [22]:
(full_diffs_df
 .fillna('nan')
 .drop_duplicates(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])
 .replace(to_replace='nan', value=np.nan)
 .to_pickle('partial-nyt-output.pkl', compression='gzip')
)

In [23]:
s3_db_dir = 's3://aspangher/edit-pathways/dbs'
s3_csv_dir = 's3://aspangher/edit-pathways/csvs'
s3_pq_dir = 's3://aspangher/edit-pathways/pqs'
s3_output_dir_main = 's3://aspangher/edit-pathways/spark_processing_scripts-output'
s3_output_dir_sentences = 's3://aspangher/edit-pathways/spark_processing_scripts-output_sentences'
pq_pat= r'df_%(news_source)s__start_\d+__end_\d+__num_\d+/'
csv_pat= r'df_%(news_source)s__start_\d+__end_\d+__num_\d+.csv.gz'

get_csv_files = lambda s3_path, news_source: list(filter(lambda y:
                                                    re.search(csv_pat % {'news_source': news_source}, y),
                                                    fs.ls(os.path.join(s3_path, news_source))
                                      ))

In [25]:
import re
num_prefetched_files = len(get_csv_files(s3_output_dir_main, 'nyt'))

In [29]:
num_prefetched_files

784

In [31]:
len(fs.ls('aspangher/edit-pathways/spark_processing_scripts-output/nyt/'))

784

In [14]:
## sentences

In [110]:
mkdir nyt_sent_output

In [111]:
for f in fs.ls('aspangher/edit-pathways/spark_processing_scripts-output_sentences/nyt'):
    f = '/'.join(f.split('/')[1:])
    fname = f.split('/')[-1]
    da.download_file('nyt_sent_output/' + fname , f)

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com
INFO:bot

In [116]:
import pandas as pd 
import pickle

# with open('nyt_sent_output/df_nyt__start_0__end_20000__num_1.pkl', 'rb') as f:
#     sent_df = pickle.load(f)
sent_df = pd.read_pickle('nyt_sent_output/df_nyt__start_0__end_20000__num_1.pkl', compression='gzip')

In [137]:
calc_entry_versions = (full_diffs_df
 .set_index('entry_id')[['version_x', 'version_y']]
 .unstack()
 .to_frame('version')
 .reset_index()
 .drop('level_0', axis=1)
 .drop_duplicates()
)

In [152]:
sent_df[['entry_id','version']].drop_duplicates().merge(calc_entry_versions, right_on=['entry_id', 'version'], left_on=['entry_id', 'version'], how='inner').shape

(66940, 2)

# Calculate number of expected

In [55]:
import sqlite3

In [56]:
con = sqlite3.connect('nyt.db')

In [99]:
full_df = pd.read_sql('select * from entryversion where num_versions > 1 and num_versions < 40 ', con=con)

In [98]:
eligible_ids = pd.read_sql('select entry_id, version from entryversion where num_versions > 1 and num_versions < 40 ', con=con)

In [64]:
expected = []
for entry_id, versions in eligible_ids.groupby('entry_id').aggregate(list)['version'].iteritems():
    for version_pair in zip(versions[:-1], versions[1:]):
        expected.append({'entry_id': entry_id, 'version_pair': version_pair})

In [66]:
expected_df = pd.DataFrame(expected)

In [68]:
expected_df.shape

(278944, 2)

# Look at fetching operation

In [91]:
sys.path.append('edit-project/spark_processing_scripts')

In [92]:
import util_general as ug

In [93]:
t = ug._download_prefetched_data_csv('nyt', False, True)

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com


  0%|          | 0/784 [00:00<?, ?it/s]

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: s3.dev.obdc.bcs.bloomberg.com


In [95]:
t.shape

(2147675, 7)

In [97]:
full_df.shape

(2238973, 7)

In [101]:
df = ug.get_rows_to_process_df(
    500, 0, t, full_df
)